In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.metrics
from pymer4.models import Lmer
import json


### Variability explained by verbs

In [2]:
properties_list = ['awareness',
       'change_of_location', 'change_of_state', 'changes_possession',
       'created', 'destroyed', 'existed_after', 'existed_before',
       'existed_during', 'exists_as_physical', 'instigation',
       'location_of_event', 'makes_physical_contact', 'manipulated_by_another',
       'predicate_changed_argument', 'sentient', 'stationary', 'volition']

roles_wide = pd.read_csv("decomp_pb_modified_sentences.csv", na_filter=False, index_col=0)
roles_wide["verb"] = roles_wide["Roleset"].apply(lambda x: x.split(".")[0])

roles_wide.head()


,Sentence.ID,Roleset,Pred.Token,Arg.Pos,Gram.Func,Split,awareness,change_of_location,change_of_state,changes_possession,...,predicate_changed_argument,sentient,stationary,volition,Sentence,Arg.Phrase,Arg.Stripped,Sentence.Stripped,Predicate,verb
0,0003_21,impose.01,7,11:1,other,train,1,1,4,1,...,5,1,1,1,"In July , the Environmental Protection Agency ...",on virtually all uses of asbestos,on all uses,"In July , the Environmental Protection Agency ...",imposed,impose
1,0003_21,impose.01,7,3:1,subj,train,5,1,3,1,...,3,1,1,5,"In July , the Environmental Protection Agency ...",the Environmental Protection Agency,the Environmental Protection Agency,"In July , the Environmental Protection Agency ...",imposed,impose
2,0003_21,impose.01,7,8:1,obj,test,1,1,3,1,...,5,1,1,1,"In July , the Environmental Protection Agency ...",a gradual ban,a ban,"In July , the Environmental Protection Agency ...",imposed,impose
3,0003_25,dump.01,1,0:1,subj,train,5,3,3,1,...,1,5,3,5,Workers dumped large burlap sacks of the impor...,Workers,Workers,"Workers dumped burlap sacks into a huge bin , ...",dumped,dump
4,0003_25,dump.01,1,2:2,obj,train,1,5,5,3,...,5,1,1,1,Workers dumped large burlap sacks of the impor...,large burlap sacks of the imported material,burlap sacks,"Workers dumped burlap sacks into a huge bin , ...",dumped,dump


In [3]:
roles_wide["pair_id"] = roles_wide["Sentence.ID"] + "_" + roles_wide["Arg.Pos"] + "_" + roles_wide["Predicate"]


In [8]:
def find_verb_var(property, verb_name, family):
    t = roles_wide.loc[roles_wide["Gram.Func"].isin(["subj", "obj"])][["pair_id", property, verb_name, "Gram.Func"]]
    t.columns = ["pair_id", "rating", "predicate", "position"]

    if family == "binomial":
        t["rating"] = (t["rating"] > 3).astype(int)
    else:
        t["rating"] = t["rating"].astype(int)

    model = Lmer(
        formula="rating ~ 1 + position * (1|predicate/pair_id)",
        data=t,
        family=family,
    )
    model.fit(optimizer='bobyqa')
    # model.fit(optimizer="Nelder_Mead")

    null_model = Lmer(
        formula="rating ~ 1 + position + (1|pair_id)",
        data=t,
        family=family,
    )
    null_model.fit(optimizer='bobyqa')
    # null_model.fit(optimizer="Nelder_Mead")

    # Extract variance components for sentence from both models
    var_pair_id_full = null_model.ranef_var.loc['pair_id']["Var"]
    var_pair_id_reduced = model.ranef_var.loc['pair_id:predicate']["Var"]

    # Compute the difference in variance explained by sentence not captured by verb
    var_from_verb = var_pair_id_full - var_pair_id_reduced

    verb_sentence_var = var_from_verb / var_pair_id_full
    return {
        "property": property,
        "sentence var from verb": verb_sentence_var,
    }



In [9]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="verb", family="gaussian"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


[1] "Model failed to converge with max|grad| = 0.0137962 (tol = 0.002, component 1)"
[2] " \n"                                                                           

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1297.0}

Log-likelihood: -14660.169 	 AIC: 29330.338

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  2.166  1.472
predicate          (Intercept)  0.943  0.971
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -15252.454 	 AIC: 30512.908

Random effects:

                 Name  Var    Std
pair_id   (Intercept)  3.0  1.732
Residual               0.0  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1297.0}

Log-likelihood: -14248.753 	 AIC: 28507.505

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  1.936  1.391
predicate          (Intercept)  0.893  0.945
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -14909.955 	 AIC: 29827.910

Random effects:

                 Name    Var    Std
pair_id   (Intercept)  2.747  1.657
Residual               0.000  0.000

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,0.278184
1,change_of_location,0.200113
2,change_of_state,NaN
3,changes_possession,0.160660
4,created,0.150489
5,destroyed,0.181374
6,existed_after,0.194198
7,existed_before,0.187719
8,existed_during,0.117347
9,exists_as_physical,0.196838


In [10]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="verb", family="binomial"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


boundary (singular) fit: see help('isSingular') 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1297.0}

Log-likelihood: -3930.593 	 AIC: 7869.186

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.000
predicate          (Intercept)  2.758  1.661

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -4444.274 	 AIC: 8894.548

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.038

No random effect correlations specified

Fixed effects:

boundary (singular) fit: see help('isSingular') 



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1297.0}

Log-likelihood: -3520.317 	 AIC: 7048.635

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.000
predicate          (Intercept)  3.447  1.857

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -4098.774 	 AIC: 8203.548

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.039

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,1.000000
1,change_of_location,0.008900
2,change_of_state,-5949.402561
3,changes_possession,0.037588
4,created,0.050523
5,destroyed,0.032423
6,existed_after,1.000000
7,existed_before,0.999867
8,existed_during,1.000000
9,exists_as_physical,1.000000


### Using Propbank for verb


In [11]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="Roleset", family="gaussian"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


Model failed to converge with max|grad| = 0.0210164 (tol = 0.002, component 1) 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1626.0}

Log-likelihood: -14486.972 	 AIC: 28983.944

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  1.957  1.399
predicate          (Intercept)  1.100  1.049
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -15252.454 	 AIC: 30512.908

Random effects:

                 Name  Var    Std
pair_id   (Intercept)  3.0  1.732
Residual               0.0  0.000

No random effect correlations specified

Fixed effects:

Model failed to converge with max|grad| = 0.0192257 (tol = 0.002, component 1) 



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1626.0}

Log-likelihood: -14068.351 	 AIC: 28146.702

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  1.736  1.318
predicate          (Intercept)  1.061  1.030
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -14909.955 	 AIC: 29827.910

Random effects:

                 Name    Var    Std
pair_id   (Intercept)  2.747  1.657
Residual               0.000  0.000

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,0.347771
1,change_of_location,0.262527
2,change_of_state,-inf
3,changes_possession,0.184105
4,created,0.176439
5,destroyed,0.196990
6,existed_after,0.233426
7,existed_before,0.224745
8,existed_during,0.155579
9,exists_as_physical,0.244427


In [12]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="Roleset", family="binomial"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


boundary (singular) fit: see help('isSingular') 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1626.0}

Log-likelihood: -3791.489 	 AIC: 7590.978

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.000
predicate          (Intercept)  3.737  1.933

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -4444.274 	 AIC: 8894.548

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.038

No random effect correlations specified

Fixed effects:

boundary (singular) fit: see help('isSingular') 



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id:predicate': 7751.0, 'predicate': 1626.0}

Log-likelihood: -3396.519 	 AIC: 6801.038

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.001
predicate          (Intercept)  4.678  2.163

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7752	 Groups: {'pair_id': 7751.0}

Log-likelihood: -4098.774 	 AIC: 8203.548

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.039

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,1.000000
1,change_of_location,-0.000013
2,change_of_state,-165.006313
3,changes_possession,0.092873
4,created,0.004179
5,destroyed,-0.022237
6,existed_after,1.000000
7,existed_before,0.999945
8,existed_during,0.068126
9,exists_as_physical,0.999999


### Using word senses for verb

In [13]:
disambiguated = pd.read_csv("../wordnet/disambiguated_sentencesV1.csv", index_col=0).set_index("Sentence.ID")
disambiguated.head()


,Sentence,Sentence.Disambiguated,disambiguated_dict
Sentence.ID,,,
0003_21,"In July , the Environmental Protection Agency ...","In July|july%1:28:00:: , the Environmental Pro...","{'July': 'july.n.01', 'Protection': 'protectio..."
0003_25,Workers dumped large burlap sacks of the impor...,Workers|worker%1:18:00:: dumped|dump%2:40:00::...,"{'Workers': 'worker.n.01', 'dumped': 'dump.v.0..."
0003_29,It has no bearing on our work force today .,It has|have%2:40:00:: no bearing|bearing%1:24:...,"{'has': 'have.v.01', 'bearing': 'bearing.n.01'..."
0003_3,Although preliminary findings were reported *-...,Although preliminary|preliminary%3:00:00:explo...,"{'preliminary': 'preliminary.s.01', 'findings'..."
0003_9,Dr. Talcott led a team of researchers from the...,Dr.|dr.%1:18:00:: Talcott led|lead%2:38:01:: a...,"{'Dr.': 'doctor.n.01', 'led': 'lead.v.01', 'te..."


In [15]:
def get_verb_sense(row):
    predicate = row["Predicate"]
    sid = row["Sentence.ID"]
    d = disambiguated.loc[sid]["disambiguated_dict"]
    # replace single with double quotes so json can parse
    d = d.replace(" '", ' "').replace("':", '":').replace("',", '",').replace("{'", '{"').replace("'}", '"}')
    try:
        d = json.loads(d)
    except:
        print(d)
    if predicate in d:
        return d[predicate]
    else:
        print(predicate, d)
        print(disambiguated.loc[sid]["Sentence.Disambiguated"])
        return np.nan

roles_wide["verb_sense"] = roles_wide.apply(get_verb_sense, axis=1)
roles_wide.dropna(subset="verb_sense", inplace=True)
roles_wide.head()


sense {'active': 'active.a.05', 'suspension': 'suspension.n.05', 'systems': 'system.n.01', 'electronically': 'electronically.r.01', 'road': 'road.n.01', 'conditions': 'weather.n.01', 'adjust': 'adjust.v.01', 'car': 'car.n.01', 'ride': 'ride.n.02'}
These `` active|active%3:00:01:: suspension|suspension%1:06:00:: systems|system%1:06:00:: '' electronically|electronically%4:02:00:: sense road|road%1:06:00:: conditions|conditions%1:19:00:: and adjust|adjust%2:30:00:: a car|car%1:06:00:: 's ride|ride%1:06:00:: . 

sense {'active': 'active.a.05', 'suspension': 'suspension.n.05', 'systems': 'system.n.01', 'electronically': 'electronically.r.01', 'road': 'road.n.01', 'conditions': 'weather.n.01', 'adjust': 'adjust.v.01', 'car': 'car.n.01', 'ride': 'ride.n.02'}
These `` active|active%3:00:01:: suspension|suspension%1:06:00:: systems|system%1:06:00:: '' electronically|electronically%4:02:00:: sense road|road%1:06:00:: conditions|conditions%1:19:00:: and adjust|adjust%2:30:00:: a car|car%1:06:00::

,Sentence.ID,Roleset,Pred.Token,Arg.Pos,Gram.Func,Split,awareness,change_of_location,change_of_state,changes_possession,...,stationary,volition,Sentence,Arg.Phrase,Arg.Stripped,Sentence.Stripped,Predicate,verb,pair_id,verb_sense
0,0003_21,impose.01,7,11:1,other,train,1,1,4,1,...,1,1,"In July , the Environmental Protection Agency ...",on virtually all uses of asbestos,on all uses,"In July , the Environmental Protection Agency ...",imposed,impose,0003_21_11:1_imposed,inflict.v.01
1,0003_21,impose.01,7,3:1,subj,train,5,1,3,1,...,1,5,"In July , the Environmental Protection Agency ...",the Environmental Protection Agency,the Environmental Protection Agency,"In July , the Environmental Protection Agency ...",imposed,impose,0003_21_3:1_imposed,inflict.v.01
2,0003_21,impose.01,7,8:1,obj,test,1,1,3,1,...,1,1,"In July , the Environmental Protection Agency ...",a gradual ban,a ban,"In July , the Environmental Protection Agency ...",imposed,impose,0003_21_8:1_imposed,inflict.v.01
3,0003_25,dump.01,1,0:1,subj,train,5,3,3,1,...,3,5,Workers dumped large burlap sacks of the impor...,Workers,Workers,"Workers dumped burlap sacks into a huge bin , ...",dumped,dump,0003_25_0:1_dumped,dump.v.01
4,0003_25,dump.01,1,2:2,obj,train,1,5,5,3,...,1,1,Workers dumped large burlap sacks of the impor...,large burlap sacks of the imported material,burlap sacks,"Workers dumped burlap sacks into a huge bin , ...",dumped,dump,0003_25_2:2_dumped,dump.v.01


In [16]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="verb_sense", family="gaussian"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


[1] "Model failed to converge with max|grad| = 0.00344168 (tol = 0.002, component 1)"
[2] " \n"                                                                            

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id:predicate': 7743.0, 'predicate': 1760.0}

Log-likelihood: -14527.083 	 AIC: 29064.165

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  1.990  1.411
predicate          (Intercept)  1.036  1.018
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 1 negative eigenvalues 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id': 7743.0}

Log-likelihood: -15236.028 	 AIC: 30480.056

Random effects:

                 Name  Var    Std
pair_id   (Intercept)  3.0  1.732
Residual               0.0  0.000

No random effect correlations specified

Fixed effects:

Model failed to converge with max|grad| = 0.0223185 (tol = 0.002, component 1) 



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position*(1|predicate/pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id:predicate': 7743.0, 'predicate': 1760.0}

Log-likelihood: -14086.392 	 AIC: 28182.784

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  1.746  1.321
predicate          (Intercept)  1.032  1.016
Residual                        0.000  0.000

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: rating~1+position+(1|pair_id)

Family: gaussian	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id': 7743.0}

Log-likelihood: -14893.895 	 AIC: 29795.791

Random effects:

                 Name    Var    Std
pair_id   (Intercept)  2.746  1.657
Residual               0.000  0.000

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,0.336549
1,change_of_location,0.253065
2,change_of_state,1.000000
3,changes_possession,0.153559
4,created,0.166658
5,destroyed,0.194514
6,existed_after,0.230920
7,existed_before,0.229316
8,existed_during,0.145940
9,exists_as_physical,0.237984


In [17]:
verb_vars = []
for property in properties_list:
    verb_vars.append(find_verb_var(property, verb_name="verb_sense", family="binomial"))

verb_vars = pd.DataFrame(verb_vars)
verb_vars


boundary (singular) fit: see help('isSingular') 



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id:predicate': 7743.0, 'predicate': 1760.0}

Log-likelihood: -3830.457 	 AIC: 7668.915

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.000
predicate          (Intercept)  3.544  1.883

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id': 7743.0}

Log-likelihood: -4439.642 	 AIC: 8885.285

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.038

No random effect correlations specified

Fixed effects:



...

/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position*(1|predicate/pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id:predicate': 7743.0, 'predicate': 1760.0}

Log-likelihood: -3412.283 	 AIC: 6832.566

Random effects:

                          Name    Var    Std
pair_id:predicate  (Intercept)  0.000  0.000
predicate          (Intercept)  4.805  2.192

No random effect correlations specified

Fixed effects:



/Users/lli/anaconda3/envs/metaphor/lib/python3.11/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: rating~1+position+(1|pair_id)

Family: binomial	 Inference: parametric

Number of observations: 7744	 Groups: {'pair_id': 7743.0}

Log-likelihood: -4094.342 	 AIC: 8194.684

Random effects:

                Name    Var    Std
pair_id  (Intercept)  0.001  0.039

No random effect correlations specified

Fixed effects:



,property,sentence var from verb
0,awareness,1.000000
1,change_of_location,0.048395
2,change_of_state,-inf
3,changes_possession,-0.003909
4,created,0.071388
5,destroyed,-0.026144
6,existed_after,1.000000
7,existed_before,0.999999
8,existed_during,1.000000
9,exists_as_physical,0.241156
